In [18]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from imblearn.combine import SMOTETomek 

from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import PrecisionRecallDisplay, precision_recall_curve

import prepare
import model

from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import label_binarize
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_selection import SelectKBest, f_regression

In [19]:
# acquire dataset
df = pd.read_csv('healthcare-dataset-stroke-data.csv')

In [20]:
# prepare data using script
df = prepare.prep_data(df).drop(columns=['age_range'])

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5109 entries, 0 to 5108
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   gender             5109 non-null   object 
 1   age                5109 non-null   float64
 2   hypertension       5109 non-null   object 
 3   heart_disease      5109 non-null   object 
 4   ever_married       5109 non-null   object 
 5   work_type          5109 non-null   object 
 6   residence_type     5109 non-null   object 
 7   avg_glucose_level  5109 non-null   float64
 8   bmi                5109 non-null   float64
 9   smoking_status     5109 non-null   object 
 10  stroke             5109 non-null   int64  
dtypes: float64(3), int64(1), object(7)
memory usage: 439.2+ KB


In [22]:
# split the dataset into train_validate and test
train_validate, test = train_test_split(df, test_size=.2, random_state=777)

In [23]:
# the second split of train and validate
train, validate = train_test_split(train_validate, test_size=.25, random_state=777)

In [24]:
# checking the shape of each dataset
train.shape, validate.shape, test.shape

((3065, 11), (1022, 11), (1022, 11))

In [37]:
# viewing only up 18 - 25 age range from the data set
mask = (train.age < 25) & (train.age >= 18)
train_subset = train[mask]
train_subset


,gender,age,hypertension,heart_disease,ever_married,work_type,residence_type,avg_glucose_level,bmi,smoking_status,stroke
531,Male,23.0,0,0,No,Private,Rural,93.74,31.2,never smoked,0
4885,Female,21.0,0,0,No,Private,Rural,55.12,21.8,never smoked,0
2820,Female,20.0,0,0,No,Private,Rural,102.42,18.6,never smoked,0
1492,Female,23.0,0,0,No,Private,Rural,65.90,21.5,never smoked,0
4417,Male,18.0,0,0,No,Private,Rural,98.07,24.0,never smoked,0
...,...,...,...,...,...,...,...,...,...,...,...
2832,Male,24.0,0,0,No,Private,Urban,84.16,37.5,smokes,0
858,Female,19.0,0,0,No,Private,Rural,77.19,37.4,smokes,0
3120,Female,23.0,0,0,No,Private,Rural,91.95,23.0,Unknown,0
4412,Male,21.0,0,0,No,Private,Urban,73.81,19.8,Unknown,0


In [38]:
# setting up for boolean masking 
mask1 = (train_subset.stroke == 0)
mask2 = (train_subset.stroke == 1)
has_stroke = train_subset[mask2]
not_stroke = train_subset[mask1]

In [39]:
# perform t-test
stats.mannwhitneyu(has_stroke.avg_glucose_level, not_stroke.avg_glucose_level, alternative = "greater")


/opt/homebrew/anaconda3/lib/python3.8/site-packages/scipy/stats/stats.py:7171: RuntimeWarning: divide by zero encountered in double_scalars
  z = (bigu - meanrank) / sd


MannwhitneyuResult(statistic=0.0, pvalue=1.0)

In [40]:
has_stroke

,gender,age,hypertension,heart_disease,ever_married,work_type,residence_type,avg_glucose_level,bmi,smoking_status,stroke


In [41]:
train_subset.stroke.value_counts()

0    212
Name: stroke, dtype: int64

### In the age group 18 to 25, there was no incident of stroke

In [30]:
# looking at 40 to 65 age group
mask3 = (train.age < 65) & (train.age >= 40)
train_subset1 = train[mask3]
train_subset1

,gender,age,hypertension,heart_disease,ever_married,work_type,residence_type,avg_glucose_level,bmi,smoking_status,stroke
4699,Male,47.0,0,0,Yes,Private,Rural,110.14,30.5,smokes,0
5035,Male,56.0,0,0,Yes,Self-employed,Rural,125.87,24.6,never smoked,0
1695,Male,57.0,0,0,Yes,Private,Rural,70.16,25.8,formerly smoked,0
1459,Female,49.0,0,0,Yes,Govt_job,Rural,83.84,19.3,formerly smoked,0
166,Female,56.0,0,0,Yes,Govt_job,Rural,162.23,27.3,Unknown,1
...,...,...,...,...,...,...,...,...,...,...,...
2156,Female,55.0,0,0,Yes,Self-employed,Urban,59.20,43.8,never smoked,0
4736,Female,54.0,0,0,Yes,Govt_job,Urban,231.54,29.9,never smoked,0
1345,Female,47.0,0,0,Yes,Private,Urban,112.09,24.7,smokes,0
2903,Male,63.0,0,0,Yes,Private,Rural,104.79,24.1,Unknown,0


In [31]:
train_subset1.stroke.value_counts()

0    1099
1      52
Name: stroke, dtype: int64

In [32]:
mask3 = (train_subset1.stroke == 0)
mask4 = (train_subset1.stroke == 1)
has_stroke = train_subset1[mask4]
not_stroke = train_subset1[mask3]

In [33]:
stats.mannwhitneyu(has_stroke.avg_glucose_level, not_stroke.avg_glucose_level, alternative = 'greater')


MannwhitneyuResult(statistic=38430.0, pvalue=1.2900358295000951e-05)

### takeaway: 
The age group 40 to 65 that had a stroke has a higer glucose level than those that did not have a stroke.
